In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime

In [ ]:
covid_confirmed_df_raw = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv')
covid_deaths_df_raw = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv')
usa_population_df_raw = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
us_state_vaccinations_df_raw = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')

In [ ]:
us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AS': 'American Samoa', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO':'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'DC': 'District of Columbia', 'FL': 'Florida', 'GA': 'Georgia', 'GU': 'Guam', 'HI': 'Hawaii',
    'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine',
    'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York State', 'NC': 'North Carolina',
    'ND': 'North Dakota', 'MP': 'Northern Mariana Islands', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'PR': 'Puerto Rico',
    'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VI': 'Virgin Islands', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming' 
}

In [ ]:
'''Cleaning the data'''
covid_confirmed_df = covid_confirmed_df_raw [covid_confirmed_df_raw['countyFIPS']>0]
covid_deaths_df = covid_deaths_df_raw [covid_deaths_df_raw['countyFIPS']>0]
usa_population_df = usa_population_df_raw [usa_population_df_raw['population']>0]
us_state_vaccinations_df = us_state_vaccinations_df_raw.fillna(axis=0, method='ffill') # handling empty slots by filling previous day data

'''Retaining some lists that are useful in further questions'''
union_territories = ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands', 'Republic of Palau', 'Federated States of Micronesia', 'Marshall Islands']

usa_population_usefacts_df = usa_population_df.copy()
usa_population_usefacts_df['state_abbrev'] = usa_population_usefacts_df['State'].map(us_state_abbrev)
us_states = usa_population_usefacts_df['state_abbrev'].unique()
usa_population_df = usa_population_usefacts_df.copy()

special_group = us_state_vaccinations_df[~us_state_vaccinations_df['location'].isin(union_territories)]
special_group = special_group[~special_group['location'].isin(us_states)]
special_group = special_group[special_group['location']!='United States']
special_group = special_group['location'].unique()

dc = 'District of Columbia'

us_states = np.delete(us_states, np.where(us_states==dc))

In [ ]:
def get_weekly_data(dataframe, first_sunday):
    weekly_counts = dataframe.loc[:,first_sunday::7]
    weekly_counts.columns = pd.to_datetime(weekly_counts.columns) #to change datatype of dates from string to datetime
    weekly_counts.columns = weekly_counts.columns.strftime('%Y/%m/%d')
    return weekly_counts

In [ ]:
def get_people_immune():
    first_sunday = '2020-01-26'
    covid_cases_df = get_weekly_data(covid_confirmed_df, first_sunday)
    covid_death_df = get_weekly_data(covid_deaths_df, first_sunday)
    people_immune = covid_cases_df - covid_death_df
    return people_immune

In [ ]:
def get_vaccination_data_for_states(dataframe):
    vacc_value_df = dataframe[dataframe['location']!='United States']
    vacc_value_df = vacc_value_df[~vacc_value_df['location'].isin(special_group)]
    vacc_value_df = vacc_value_df[~vacc_value_df['location'].isin(union_territories)]
    return vacc_value_df

In [ ]:
def plot(dataframe):
    plt.figure(figsize=(20,8))
    plt.xticks(rotation=70)
    sns.lineplot(data=dataframe)

# Q2.The vaccination data is reported by the entire county, state, US territory, and various groups like the department of defense, long term care and others. Presumably the US data includes the state data and the special group data but not the US territory data. Is the US territory data included in the US totals. Justify your answer.

In [ ]:
us_state_vaccinations_total_df = us_state_vaccinations_df.groupby(['location']).tail(1)

united_states_count_df = us_state_vaccinations_total_df[us_state_vaccinations_total_df['location']=='United States']

only_states_df = get_vaccination_data_for_states(us_state_vaccinations_total_df)

only_ut = us_state_vaccinations_total_df[us_state_vaccinations_total_df['location'].isin(union_territories)]
only_sg = us_state_vaccinations_total_df[us_state_vaccinations_total_df['location'].isin(special_group)]

total_vaccinations_usa_states = only_states_df['total_distributed'].sum()
us_totals = united_states_count_df['total_distributed'].to_string(index=False)

print("Total Vaccination distributed over all states in USA including District of Columbia: {}".format(total_vaccinations_usa_states))
print("Total Vaccination distributed value for us_totals or location mentioned as United States in vaccination data: {}".format(us_totals))

In [ ]:
# state data with union territory data including DC
states_with_ut_and_dc = only_ut['total_distributed'].sum()+total_vaccinations_usa_states
states_with_ut_and_dc

In [ ]:
# state data with special group data including DC
states_with_sg_and_dc = only_sg['total_distributed'].sum()+total_vaccinations_usa_states
states_with_sg_and_dc

In [ ]:
only_states_df = only_states_df[only_states_df['location']!=dc]
total_vaccinations_usa_states = only_states_df['total_distributed'].sum()

# state data with union territory data excluding DC
states_with_ut_without_dc = only_ut['total_distributed'].sum()+total_vaccinations_usa_states
states_with_ut_without_dc

In [ ]:
# state data with special group data excluding DC
states_with_sg_without_dc = only_sg['total_distributed'].sum()+total_vaccinations_usa_states
states_with_sg_without_dc

Checking "total_distributed" column as it says that it is cumulative counts of COVID-19 vaccine doses recorded as shipped in CDC's Vaccine Tracking System.

According to certain combinations tried above, we can conclude that us_totals include count of vaccinations of all different states including "District of Columbia" and union territories, but it does not include any special group count.

# Q3.We would like to compute the percent of the population in each state that has been fully vaccinated. Presumably the data reported got the various groups is not included in the individual state data. If we distributed the number of people fully vaccinated in those groups to the states proportionally by population how much would that change the percent of people vaccinated in each state?

In [ ]:
statewise_population_usa = usa_population_df.groupby(['state_abbrev']).sum(['population']).drop(['countyFIPS'], axis=1).reset_index()

us_state_vaccinations_total = us_state_vaccinations_df.groupby(['location']).tail(1)
only_states_vacc_df = get_vaccination_data_for_states(us_state_vaccinations_total)
only_states_vacc_df = only_states_vacc_df[only_states_vacc_df['location']!=dc]

special_group_vacc_df = us_state_vaccinations_df[us_state_vaccinations_df['location'].isin(special_group)]
special_group_vacc_df = special_group_vacc_df.groupby(['location']).tail(1)

ppl_fully_vacc_spc_group = special_group_vacc_df['people_fully_vaccinated'].sum()

total_population_usa = statewise_population_usa['population'].sum()

statewise_pop_vacc_df = pd.merge(only_states_vacc_df, statewise_population_usa, left_on='location', right_on='state_abbrev', how='inner')

Q3_final_df = statewise_pop_vacc_df[['date','location','people_fully_vaccinated','people_fully_vaccinated_per_hundred','population']]
Q3_final_df_copy = Q3_final_df.copy()
Q3_final_df_copy['change'] = (Q3_final_df_copy.loc[:,'population'].div(total_population_usa)).mul(ppl_fully_vacc_spc_group)
Q3_final_df_copy['new_ppl_vacc_per_100'] = ((Q3_final_df_copy.loc[:,'change']+Q3_final_df_copy.loc[:,'people_fully_vaccinated']).div(Q3_final_df_copy.loc[:,'population'])).mul(100)
Q3_final_df_copy['%change'] = Q3_final_df_copy['new_ppl_vacc_per_100'] - Q3_final_df_copy['people_fully_vaccinated_per_hundred']
Q3_final_df = Q3_final_df_copy

display(Q3_final_df)

# Q4. Produce a chart or table showing the percent of people in each state that have been fully vaccinated. Sort the data by the percent of people vaccinated. You notebook should download the dataset so when it is run we get the most recent data.

In [ ]:
us_pop_vacc_df = statewise_pop_vacc_df[['location','people_fully_vaccinated_per_hundred']]
us_pop_vacc_df = us_pop_vacc_df.rename(columns={'people_fully_vaccinated_per_hundred':'%_people_vaccinated'})
us_pop_vacc_df = us_pop_vacc_df[['location','%_people_vaccinated']].set_index('location').sort_values(by=['%_people_vaccinated'], ascending=False)

display(us_pop_vacc_df)

# Q5.We want to see the progression from the start of the pandemic to now the percent of people that have some immunity in the US. Produce a plot of weekly data that contains three lines. One showing the the percent of the living people have have or had covid. One showing the the percent of people that have had at least one covid vaccine shot. The third line showing the sum of the two to estimate the number of people that have some immunity. Over time sum will be an over estimate as people who had covid get a vaccine shot. As in #4 the notebook should download the data to up-to-date results.

In [ ]:
people_immune = get_people_immune()
dates = people_immune.columns

vacc_value_df = get_vaccination_data_for_states(us_state_vaccinations_df)

vacc_value_df = vacc_value_df.groupby(['date']).sum('people_vaccinated').reset_index()
vacc_value_df = vacc_value_df[['date','people_vaccinated']]
vacc_value_df = vacc_value_df.set_index('date').transpose()

first_sunday = '2021-01-17'
vacc_value_df = get_weekly_data(vacc_value_df, first_sunday)
vacc_value_df = vacc_value_df.transpose()

total_people_immune = people_immune.sum().to_frame().rename_axis("week").reset_index()
total_people_immune = total_people_immune.rename(columns={0:"people_immune"})

vacc_value_df = vacc_value_df.reindex(dates, fill_value=0).rename_axis("week").reset_index()

covid_immunity_df = total_people_immune.merge(vacc_value_df, on='week')
covid_immunity_df['sum_immune'] = covid_immunity_df['people_immune'] + covid_immunity_df['people_vaccinated']
covid_immunity_df = covid_immunity_df.set_index('week')
covid_immunity_df_copy = covid_immunity_df.copy()
covid_immunity_df = (covid_immunity_df_copy.div(total_population_usa)).mul(100)
covid_immunity_df = covid_immunity_df.rename(columns={"people_immune":"%_people_immune_recovered", "people_vaccinated":"%_people_vaccinated","sum_immune":"%_total_immunity"})

display(covid_immunity_df)
plot(covid_immunity_df)

# Q6. Produce the same plot as in #5 for the states. You should have a function that takes the two letter abbreviation for the state and produces the plot for the state.

In [ ]:
def statewise_immunity_plot_state(state_abbrev, vacc_value_df):
    first_sunday = '2021-01-17'
    state = us_state_abbrev[state_abbrev]
    state_population = statewise_pop_vacc_df[statewise_pop_vacc_df['location']==state].population.sum() #To reatin population of required state

    vacc_value_df = vacc_value_df[vacc_value_df['location']==state].set_index('date')
    vacc_value_df = vacc_value_df[['people_vaccinated']].transpose()
    vacc_value_df = get_weekly_data(vacc_value_df, first_sunday)
    vacc_value_df = vacc_value_df.transpose().reindex(dates, fill_value=0).reset_index()
    vacc_value_df = vacc_value_df.rename(columns={'index':"week"})

    people_immune = get_people_immune()
    states = covid_confirmed_df[['State']]
    people_immune['State'] = states['State']
    people_immune = people_immune.groupby(['State']).sum()
    people_immune = people_immune.loc[state_abbrev,:].transpose().reset_index()
    people_immune = people_immune.rename(columns={'index':'week', state_abbrev:'people_immune_to_covid'})

    covid_immunity_df_state = people_immune.merge(vacc_value_df, on='week')
    covid_immunity_df_state['sum_immunity'] = covid_immunity_df_state['people_vaccinated'] + covid_immunity_df_state['people_immune_to_covid']
    covid_immunity_df_state = covid_immunity_df_state.set_index('week')
    covid_immunity_df_copy = covid_immunity_df_state.copy()
    covid_immunity_df_state = (covid_immunity_df_copy.div(state_population)).mul(100)
    covid_immunity_df_state = covid_immunity_df_state.rename(columns={"people_immune_to_covid":"%_people_immune", "people_vaccinated":"%_people_vaccinated","sum_immunity":"%_total_immunity"})
    display(covid_immunity_df_state)
    plot(covid_immunity_df_state)

In [ ]:
def statewise_immunity_plot(state_abbrev):
    vacc_value_df = get_vaccination_data_for_states(us_state_vaccinations_df)
    statewise_immunity_plot_state(state_abbrev, vacc_value_df)

In [ ]:
statewise_immunity_plot('UT')

# Q1.In us_state_vaccinations.csv there are some gaps in the data. Will those gaps affect the results of the following questions. If no why not. If yes explain the affect.

Gaps would have affected if we dropped the location which had null values. 
This effects questions 2 through 4 if sunday date has empty values in it, as we are taking week counts on every sunday's.
For question 5 and 6, it would have affected if present day value is empty.

We could overcome this issue by filling those gaps with previous date's data and this resolved our problem as the data is cumulative.

us_state_vaccinations_df_raw is vaccination dataframe before forward fill.
us_state_vaccinations_df is vaccination dataframe after forward fill.

All the above questions used us_state_vaccinations_df as a vaccination dataframe.

Let's check how us_state_vaccinations_df_raw would have affected the output of Q6 for example,

In [ ]:
def statewise_immunity_plot_raw(state_abbrev):
    vacc_value_df = get_vaccination_data_for_states(us_state_vaccinations_df_raw)
    statewise_immunity_plot_state(state_abbrev, vacc_value_df)

In [ ]:
statewise_immunity_plot_raw('UT')